# 2022 Capstone Project 
# Where can soil moisture improve rainfall-triggered landslide predictability?
# Team: Jacquelyn Witte

In [11]:
import os

import earthpy as et
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from geopandas import GeoDataFrame
from shapely.geometry import Point

import warnings
warnings.filterwarnings('ignore')

sns.set(font_scale=1.2, style="whitegrid")

## Why do we care about landslides?


![I-70 mudslide in Colorado](images/cnn_newsarticle_glenspringsLandslideCO.png)

Landslides have significant economic impacts. There can be loss of life, wildlife and wildlife habitat. Landslides are a serious geologic hazard common to many areas in the US and Canada. In Colorado, which is where I live, a high percentage of our population lives in the Rockies. So transportation and infrastructure are impacted as you can see from the CNN caption above of a mudslide event on the I-70 which is our main highway artery across the Rockies.

On another note, my husband I were driving home from the Oregon coast during an Atmospheric river event and there were crews along the valley roads building retaining walls to prevent landslides because of the soil erosion caused by wildfires in that region. So climate change is also factor and one which confounds our ability to predict rainfall trigger landslides. Climate change is a chaotic system and the basis of all models is pattern and order. 

This is where soil moisture and composition and knowing the history of the area and land-uses come into play in this project. 

Landslides have multiple causes:  rainfall, snowmelt, erosion, earthquakes, volcanoes, disturbance by human activities, or any combination of these and other factors. This study focuses on rainfall-triggered landslides. Predicting them with a high degree of accuracy to save lives and maintain infrastructure is challenging. Precipitation data sources are heterogeneous - radar, satellite, in-situ and ensemble model predictions using available precipitation products carry large uncertainties (https://onlinelibrary.wiley.com/doi/full/10.1002/hyp.14260). This make it challenging to predict the timing, severity, extent, and even likelihood that a landslide will occur. 

## Motivation

Soil moisture can be a potential indicator of the type of rainfall induced landslide. We can use soil moisture data to group landslides into two types (1) shallow slope failures - saturation induced by rainfall infiltration and (2) run off driven landslides - triggered by intense storms.  This informs the forecaster which models and precipitation products are optimal to use to better predict a landslide event.

The purpose of this study is to address the need to improve our understanding of landslides caused by rainfall. To seek answers to the question of where and when will landslides occur?

This work follows on research done by Elsa Culler and colleagues in their paper:
> Culler, E. S., Badger, A. M., Minear, J. T., Tiampo, K. F., Zeigler, S. D., & Livneh, B. (2021). A multi-sensor evaluation of precipitation uncertainty for landslide-triggering storm events. Hydrological Processes, 35( 7), e14260. https://doi.org/10.1002/hyp.14260

## Study region: Colorado, USA

The study area will be Colorado, because I live here and am somewhat familiar with the landscape. 

Towns and cities in mountainous or hilly terrain (think Colorado or California) are growing and spreading as populations move into these popular destination areas. It is important to understand the potential exposure to landslide hazards, and how counties can maintain optimal land-use and infrastructure to reduce the costs of living with landslides. 

Below is map of the Colorado with all the Landslide events between 2007-2016. The locations are color coated by trigger - Continuous rain, downpour, rain, tropical cyclone which we have one documented and unknown.
> Data Source: Global Landslide Catalog https://doi.org/10.1002/hyp.14260

In [15]:
# Change to data directory
data_dir = os.path.join(et.io.HOME,
                        'earth-analytics',
                        'data')
os.chdir(data_dir)
# Read Global Landslide Catalog CSV to a dataFrame
glc = pd.read_csv(os.path.join('capstone', 
                               'landslide', 
                               'nasa.glc.export.csv'))
# Subset to Colorado and remove non-rainfall events
glc_co = glc[(glc['country_code'] == 'US') & (
                                glc['admin_division_name'] == 'Colorado') & (
                                glc['landslide_trigger'] != 'freeze_thaw') &(
                            glc['landslide_trigger'] != 'snowfall_snowmelt')
            ]
# covert to datetime
glc_co['date'] = pd.to_datetime(glc_co['event_date'])
glc_co = glc_co.set_index('date').sort_index()

# georeference the locations to plot on Folium
geometry = [Point(xy) for xy in zip(glc_co['longitude'], glc_co['latitude'])]
crs = 'epsg:4326'
glc_co_epsg4326 = GeoDataFrame(glc_co, crs=crs, geometry=geometry)

# Create a symbology dictionary that maps "type" to color. 
triggers = np.unique(glc_co_epsg4326['landslide_trigger'])
colors = ['red', 'green', 'blue', 'orange','lightgray']
points_symb = dict(zip(triggers,colors))

# map the Landslide events
# Create Title
loc = 'Rainfall-triggered Landslides in Colorado (2007 - 2016)'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m2 = folium.Map([39.0, -105.27], zoom_start=7, tiles='Stamen Terrain')

for i, loc in glc_co_epsg4326.iterrows():
    folium.Marker(location=[loc['latitude'], loc['longitude']],
                  icon=folium.Icon(color=points_symb[loc['landslide_trigger']],
                                   prefix='glyphicon', icon='off')).add_to(m2)

# Continuous Rain
folium.CircleMarker(location=[40.5, -103], radius=10,
                    color='red',
                    fill_color='red',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[40.5, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Continuous Rain'}</b>""")
).add_to(m2)

# Downpour
folium.CircleMarker(location=[40, -103], radius=10,
                    color='green',
                    fill_color='green',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[40, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Downpour'}</b>""")
).add_to(m2)

# Rain
folium.CircleMarker(location=[39.5, -103], radius=10,
                    color='blue',
                    fill_color='blue',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[39.5, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Rain'}</b>""")
).add_to(m2)
   
    
# Tropical Cyclone
folium.CircleMarker(location=[39., -103], radius=10,
                    color='orange',
                    fill_color='orange',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[39., -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Tropical Cyclone'}</b>""")
).add_to(m2)
   
# Unknown
folium.CircleMarker(location=[38.5, -103], radius=10,
                    color='lightgray',
                    fill_color='lightgray',
                    fill_opacity=1.0).add_to(m2)
folium.Marker(
    location=[38.5, -102.7],
    icon=folium.DivIcon(
        html=f"""<b>{'Unknown'}</b>""")
).add_to(m2)

    
#Add Title
m2.get_root().html.add_child(folium.Element(title_html))
m2

## Data Sources